In [1]:
from Layers import Node, Source, Sink, PHY, Channel
from DataLink import *
from simulator import Simulator, Event_queues
from CRC import CRC

In [2]:
# Auto-reload extension (just for dev phase)\n",
%load_ext autoreload
%autoreload 2

# Create simulator

In [43]:
p = 1e-4
R_T = 1e6
D_p = 1e-3

divisor = 0xB
checker = CRC(divisor)

timeout = (2*D_p + 9/R_T)*1.01
packet_size = 96

In [66]:
simulator = Simulator()
# Nodes
node_tx = Node(simulator)
node_rx = Node(simulator)
# Channel: 2 same channel
## create
chann_tx = Channel(p, R_T, D_p)
chann_rx = Channel(p, R_T, D_p)
## bind to simulator
chann_tx.bind_sim(simulator)
chann_rx.bind_sim(simulator)
# Add channels, layers to nodes
## Tx
node_tx.bind_channels(chann_tx, chann_rx)
node_tx.bind_element('L1', PHY(checker=checker))
node_tx.bind_element('L2', Stop_Wait_Tx(timeout))
node_tx.bind_element('L3', Source(packet_size))
## Rx
node_rx.bind_channels(chann_rx, chann_tx)
node_rx.bind_element('L1', PHY(checker=checker))
node_rx.bind_element('L2', Stop_Wait_Rx())
node_rx.bind_element('L3', Sink())

## Add initializing event
In this excersice we add `start transmit`.

In [67]:
node_tx.event_queue.clear_event()

event_init = (0, 'DL start', None)
node_tx.add_event(event_init)

node_tx.event_queue

[(0, 'DL start', None)]

# Run simulator

In [68]:
t_end = 10
simulator.run(t_end)

In [69]:
D_T = (packet_size+4)/R_T
utilization = (node_rx.L3.counter * D_T) / t_end
print(f'practical:\t{utilization}')
print(f'theory ideal:\t{D_T/(D_T + 2 * D_p)}')
print(f'theory noisy:\t{(D_T/(D_T + 2 * D_p) * (1-p*(packet_size+4)))}')

practical:	0.04685
theory ideal:	0.04761904761904762
theory noisy:	0.047142857142857146
